### Unit 1 Homework:  Scraping the Yelp Website

Welcome!  For this homework assignment you'll be tasked with building a web scraper in a manner that builds on what was covered in our web scraping class.

The assignment will extend the lab work done during that time, where we built a dataset that listed the name, number of reviews and price range for restaurant on the following web page: https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1

**What You'll Turn In:**

A finished jupyter notebook that walks us through the steps you took in order to get your results.  Provide notes where appropriate to explain what you are doing.

The notebook should produce a finished dataset at the end.  

If for some reason you're experiencing problems with the final result, please let someone know when turning it in.
 
The homework is divided into five tiers, each of which have increasing levels of difficulty:

##### Tier 1: Five Columns From the First Page

At the most basic level for this assignment, you will need to extend what we did in class, and create a dataset that has five columns in it that are 30 rows long.  This means you will not need to go off the first page in order to complete this section.

##### Tier 2:  100 Row Dataset With At Least 3 Columns

For this portion of the assignment, take 3 of your columns from step 1, and extend them out to multiple pages on the yelp website.  You should appropriately account for the presence of missing values.

##### Tier 3:  100 Row Dataset With At Least 5 Columns

Very similar to Tier 2, but if you use this many columns you will be forced to encounter some columns that will frequently have missing values, whereas with Tier 2 you could likely skip these if you wanted to.  

##### Tier 4:  100 Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Restaurants often have different categories associated with them, so grabbing them individually as separate values is often challenging.  To complete this tier, you'll have to find a way to 'pick out' each of the individual categories as their own separate column value.  

##### Tier 5:  Unlimited Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Take what you did in Tier 4, and extend it so that the code will work with an arbitrary number of pages.  Ie, regardless of how many pages there are listing the best restaurants in London, your scraper will find them, and cleanly parse their information into clean datasets.

### Hints

Here are a few tips that will save you time when completing this assignment:

 - The name, average rating, total ratings and neighborhood of a restaurant tend to be the 'easy' ones, because they rarely have missing values, so what ever logic you use on the first page will typically apply to all pages.  They are a good place to start
 - Phone numbers, price ranges and reviews are more commonly missing, so if you are trying to get a larger number of items from them across multiple pages you should expect to do some error handling
 - You can specify any sort of selector when using the `find_all()` method, not just `class`.  For example, imagine you have the following `<div>` tag:
    `<div class='main-container red-blue-green' role='front-unit' aria-select='left-below'>Some content here</div>`
    
   This means that when you use `scraper.find_all('div')`, you can pass in arguments like `scraper.find_all('div', {'role': 'front-unit'})` or anything else that allows you to isolate that particular tag.
 - When specifying selectors like `{'class': 'dkght__384Ko'}`, sometimes less is more.  If you include multiple selectors, you are saying return a tag with **any one of these** distinctions, not all of them.  So if your results are large, try different combinations of selectors to get the smallest results possible.
 - If you begin dealing with values that are unreliably entered, you should use the 'outside in' technique where you grab a parent container that holds the element and find a way to check to see if a particular value is there by scraping it further.  The best way to do this is to try and find a unique container for every single restaurant.  This means that you will have a reliable parent element for every single restaurant, and within *each of these* you can search for `<p>`, `<a>`, `<div>`, and `<span>` tags and apply further logic.
 - When you get results from `BeautifulSoup`, you will be given data that's denoted as either `bs4.element.Tag` or `bs4.element.ResultSet`.  They are **not the same**.  Critically, you can search a `bs4.element.Tag` for further items, but you cannot do this with a `bs4.element.ResultSet`.  
 
   For example, let's say you grab all of the divs from a page with `scraper.find_all('div')` and save it as the variable `total_divs`.  This means `total_divs` will look somethig like this:  
   
   `[<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>]`
      
   In this case the variable `total_divs` is a result set and there's nothing else you can do to it directly.  However, every item within `total_divs` is a tag, which means you can scrape it further.  
   
   So if you wanted you could write a line like:  `total_paragraphs = [div.find_all('p') for div in total_divs]`, and get the collection of paragraphs within each div.  
   
   If you confuse the two you'll get the following error message:  
   
   `AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?`
 - The values of the different selectors change periodically on yelp, so if your scraper all of a sudden stops working that's probably why.  Ie, if you have a command like `scraper.find_all('div', {'class': '485dk0W__container09'}` that no longer returns results, the class `485dk0W__container09` may now be `r56kW__container14` or something similar.

First I am going to grab the yelp website's HTLM to begin scriping data. 

In [1]:
#import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
from re import search

In [3]:
all_pages = [0, 10, 20, 30, 40, 50, 60, 70, 80, ]


#Number of reviews
n_reviews_messy = scraper.find_all('span', {'class': 'reviewCount__09f24__EUXPN'})
n_reviews = [int(review.text) for review in n_reviews_messy]


In [52]:
#Price Range 
price_range_messy = scraper.find_all('span', {'class': 'priceRange__09f24__2O6le', 'class': 'css-xtpg8e'})
price_range = [price.text for price in price_range_messy]

In [181]:
price_ranges_final = []
titles_final       = []
n_reviews_final    = []
neighborhood_final = []
ratings_final      = []

for i in range(0, 120, 10): 
    url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=11&start={i}'
    req_yelp = requests.get(url).text #request to get to the URL 
    scraper = BeautifulSoup(req_yelp)
    
    ###Get containers so we can get missing values easier###
    containers = scraper.find_all('div','container__09f24__21w3G')

    
    ### Price Ranges ###
    pr_messy = [container.find_all('span')[5].text for container in containers]
    pr = [price if len(price) < 6 else None for price in pr_messy]
    price_ranges_final += pr
    
    
    ### Restaruant Titles ###
    titles = [container.find_all('a')[1].text for container in containers]
    titles_final += titles
    
    
    ###Number of Reviews ###
    n_reviews_messy = scraper.find_all('span', {'class': 'reviewCount__09f24__EUXPN'})
    n_reviews = [int(review.text) for review in n_reviews_messy]
    n_reviews_final += n_reviews
    
    
    ### Neighborhood ###
    nb_messy = [container.find_all('p')[3].text for container in containers]
    nb = [l if len(l) < 20 else None for l in nb_messy]  
    neighborhood_final += nb
    
    
    ### Average Ratings ###
    avg_rating_messy = scraper.find_all('div', {'class': 'i-stars__09f24__1T6rz'})
    ratings_text = [rating["aria-label"] for rating in avg_rating_messy]
    ratings = [rating.replace(' star rating', '') for rating in ratings_text]
    ratings_final += ratings


In [182]:
### Create data Frame ###
yelp_dict = {
'name': titles_final,
'n_reviews': n_reviews_final,
'price_range': price_ranges_final, 
'avg_rating': ratings_final, 
'neighborhood': neighborhood_final
}


df = pd.DataFrame(yelp_dict)

In [183]:
df

,name,n_reviews,price_range,avg_rating,neighborhood
0,The Mayfair Chippy,282,££,4.5,Mayfair
1,Dishoom,1841,££,4.5,Covent Garden
2,The Breakfast Club,494,££,4,Soho
3,Flat Iron,380,££,4.5,None
4,Ffiona’s Restaurant,267,££,4.5,Kensington
...,...,...,...,...,...
115,Dalloway Terrace,34,£££,4.5,Bloomsbury
116,Laughing Halibut,242,££,4,Westminster
117,The Sandwich Centre,13,£,4.5,St James's
118,Wok to Walk - Oxford St,16,£,4.5,Soho


In [179]:
#Attempt at getting the restaurant categories. Got stuck and was unable to complete 
#(but wanted to include these bits of code since I spent a lot of time on them)



url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=11&start=0'
req_yelp = requests.get(url).text #request to get to the URL 
scraper = BeautifulSoup(req_yelp)
containers = scraper.find_all('div','container__09f24__21w3G')
tags_messy = [container.find_all('span') for container in containers]
tags = [a[7].text for a in tags_messy]
cats = []
for cat in tags:
    if cat.count(",") == 0:
        one = cat + ', None' + ', None'
    elif cat.count(",") == 1: 
        two = cat + ', None'
    elif cat.count(",") == 2:
        three = cat

#I was attempting to do Teir Four but I got stuck. I tried to apply the above logic but realized it 
#wasn't going to work because there was not way to keep the original ordering and put the categories
#back with the same restaurant info. Interested in seeing how this is done in class on Monday

Coffee & Tea, Breakfast & Brunch, American (Traditional)
American (Traditional), Soul Food, Cajun/Creole
